## Importing Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Importing table from webpage to a Dataframe

In [2]:
# retreiving html
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')

# retreiving the whole table
table_rows = soup.find('table').find_all('tr')

# headers are in the item of the list
headers = table_rows[0].find_all('th') 
columns = [i.text.replace('\n','') for i in headers]

# retreiving each row
rows = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text.replace('\n','') for i in td]
    rows.append(row)
# first row is empty because of the headers
rows = rows[1:len(rows)]

# importing the raw data into a dataframe
data_frame = pd.DataFrame(rows, columns = columns)

data_frame.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


## Cleaning Data

In [13]:
# removing Borough 'Not assigned'
data_frame = data_frame[data_frame['Borough']!='Not assigned']
data_frame.reset_index(inplace = True, drop = True)

# replacing '/' with ',' to sepatate neighborhoods in the same Postal Code
for i in range(data_frame.shape[0]):
    data_frame['Neighborhood'][i] = data_frame['Neighborhood'][i].replace(' /',',') 
 
print("Number of rows in the data_frame = {}".format(data_frame.shape[0]))
data_frame.head(10)

Number of rows in the data_frame = 103


,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


## Getting the geographical coordinates for each postal code in Toronto

In [28]:
# I decided to import from the csv file because the geocoder package was not working
file = 'Geospatial_Coordinates.csv'
lat_lon = pd.read_csv(file)
lat_lon

# merging dataframes keeping only the Postal Codes that are available in the table extracted from wikipedia
toronto_neigh = pd.merge(data_frame, lat_lon, left_on='Postal code', right_on='Postal Code', how = 'left')
toronto_neigh.drop('Postal Code', axis = 1, inplace = True)
toronto_neigh.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
